# **Data Cleaning and Merging**

In [1]:
import pandas as pd 
import sqlite3 as sqlite
conn = sqlite.connect('motor_vehicle_collisions.db')


## **_Data Import, Selection and Review_**

In [2]:
mvc_df = pd.read_csv('Motor_Vehicle_Collisions_-_Crashes.csv', low_memory=False)
mvc_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2085707 entries, 0 to 2085706
Data columns (total 29 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   CRASH DATE                     object 
 1   CRASH TIME                     object 
 2   BOROUGH                        object 
 3   ZIP CODE                       object 
 4   LATITUDE                       float64
 5   LONGITUDE                      float64
 6   LOCATION                       object 
 7   ON STREET NAME                 object 
 8   CROSS STREET NAME              object 
 9   OFF STREET NAME                object 
 10  NUMBER OF PERSONS INJURED      float64
 11  NUMBER OF PERSONS KILLED       float64
 12  NUMBER OF PEDESTRIANS INJURED  int64  
 13  NUMBER OF PEDESTRIANS KILLED   int64  
 14  NUMBER OF CYCLIST INJURED      int64  
 15  NUMBER OF CYCLIST KILLED       int64  
 16  NUMBER OF MOTORIST INJURED     int64  
 17  NUMBER OF MOTORIST KILLED      int64  
 18  CO

In [3]:
mvc_df = (mvc_df.copy()
          .loc[(#mvc_df['CRASH DATE'].str.contains(('2020'), regex=False, na=False) |
                #mvc_df['CRASH DATE'].str.contains(('2021'), regex=False, na=False) |
                mvc_df['CRASH DATE'].str.contains(('2022'), regex=False, na=False) |
                mvc_df['CRASH DATE'].str.contains(('2023'), regex=False, na=False)
                )]
          )


Here's a brief description of the columns in the New York City Motor Vehicle Collisions data:

1. **CRASH DATE**: The date when the collision occurred.
2. **CRASH TIME**: The time of day when the collision happened.
3. **BOROUGH**: The borough (e.g., Manhattan, Brooklyn, Queens, etc.) where the collision occurred.
4. **ZIP CODE**: The ZIP code of the collision location.
5. **LATITUDE** and **LONGITUDE**: The geographical coordinates (latitude and longitude) of the collision site.
6. **LOCATION**: A textual description of the collision location.
7. **ON STREET NAME**: The name of the street where the collision occurred.
8. **CROSS STREET NAME**: The name of the intersecting street.
9. **OFF STREET NAME**: If the collision didn't happen on a street, this field provides an off-street location description.
10. **NUMBER OF PERSONS INJURED**: The count of people injured in the collision.
11. **NUMBER OF PERSONS KILLED**: The count of fatalities resulting from the collision.
12. **NUMBER OF PEDESTRIANS INJURED**: The number of pedestrians injured.
13. **NUMBER OF PEDESTRIANS KILLED**: The number of pedestrians killed.
14. **NUMBER OF CYCLIST INJURED**: The count of cyclists injured.
15. **NUMBER OF CYCLIST KILLED**: The number of cyclists killed.
16. **NUMBER OF MOTORIST INJURED**: The count of motorists (vehicle occupants) injured.
17. **NUMBER OF MOTORIST KILLED**: The number of motorists killed.
18. **CONTRIBUTING FACTOR VEHICLE 1**, **2**, **3**, **4**, **5**: Factors contributing to the collision (e.g., speeding, distracted driving, weather conditions, etc.).
19. **COLLISION_ID**: A unique identifier for each collision record.
20. **VEHICLE TYPE CODE 1**, **2**, **3**, **4**, **5**: Descriptions of the vehicle types involved in the collision (e.g., sedan, SUV, truck, etc.).

This data is collected by the New York City Police Department (NYPD) and provides valuable insights for analyzing traffic safety and identifying hazardous intersections. You can find more details about this dataset on the [NYC Open Data website](https://data.cityofnewyork.us/Public-Safety/new-collision-data/q42n-de7v)

## **Data Wrangling**

In [4]:
## Changing the column characters
mvc_df.columns = mvc_df.columns.str.title().str.replace('_',' ')


In [5]:
mvc_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 200467 entries, 1 to 2085624
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Crash Date                     200467 non-null  object 
 1   Crash Time                     200467 non-null  object 
 2   Borough                        134498 non-null  object 
 3   Zip Code                       134474 non-null  object 
 4   Latitude                       184238 non-null  float64
 5   Longitude                      184238 non-null  float64
 6   Location                       184238 non-null  object 
 7   On Street Name                 145419 non-null  object 
 8   Cross Street Name              95605 non-null   object 
 9   Off Street Name                55045 non-null   object 
 10  Number Of Persons Injured      200467 non-null  float64
 11  Number Of Persons Killed       200467 non-null  float64
 12  Number Of Pedestrians Injured  200

### **_Data Enrichment_**

In [6]:
mvc_df['Number Of Vehicles'] = (mvc_df[['Vehicle Type Code 1',
                                        'Vehicle Type Code 2',
                                        'Vehicle Type Code 3',
                                        'Vehicle Type Code 4',
                                        'Vehicle Type Code 5'
                                        ]]
                                .notnull()
                                .sum(axis=1)
                                )


In [7]:
print(mvc_df.info())
mvc_df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 200467 entries, 1 to 2085624
Data columns (total 30 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Crash Date                     200467 non-null  object 
 1   Crash Time                     200467 non-null  object 
 2   Borough                        134498 non-null  object 
 3   Zip Code                       134474 non-null  object 
 4   Latitude                       184238 non-null  float64
 5   Longitude                      184238 non-null  float64
 6   Location                       184238 non-null  object 
 7   On Street Name                 145419 non-null  object 
 8   Cross Street Name              95605 non-null   object 
 9   Off Street Name                55045 non-null   object 
 10  Number Of Persons Injured      200467 non-null  float64
 11  Number Of Persons Killed       200467 non-null  float64
 12  Number Of Pedestrians Injured  200

,Crash Date,Crash Time,Borough,Zip Code,Latitude,Longitude,Location,On Street Name,Cross Street Name,Off Street Name,...,Contributing Factor Vehicle 3,Contributing Factor Vehicle 4,Contributing Factor Vehicle 5,Collision Id,Vehicle Type Code 1,Vehicle Type Code 2,Vehicle Type Code 3,Vehicle Type Code 4,Vehicle Type Code 5,Number Of Vehicles
1,03/26/2022,11:45,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,...,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN,1
2,06/29/2022,6:55,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,...,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN,2
34,06/29/2022,16:00,NaN,NaN,NaN,NaN,NaN,WILLIAMSBURG BRIDGE OUTER ROADWA,NaN,NaN,...,NaN,NaN,NaN,4542336,Motorscooter,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,2
37,07/12/2022,17:50,BROOKLYN,11225,40.663303,-73.96049,"(40.663303, -73.96049)",NaN,NaN,44 EMPIRE BOULEVARD,...,NaN,NaN,NaN,4545699,Sedan,NaN,NaN,NaN,NaN,1
38,03/23/2022,10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71 EAST DRIVE,...,NaN,NaN,NaN,4512922,Bike,NaN,NaN,NaN,NaN,1


### **_Creating Related Tables to Feed into the Relationship Database_**

In [9]:
## Vehicle collision data 
vehicle_collisions = (mvc_df[['Crash Date', 'Crash Time', 'Collision Id',
                              'Borough','Latitude','Longitude','Zip Code',
                              'On Street Name',
                              'Off Street Name','Cross Street Name'
                              ]]
                      .rename(columns = {'Crash Date':'date',
                                         'Crash Time':'time',
                                         'Collision Id':'collision_id',
                                         'Zip Code':'zip_code',
                                         'Borough':'Town',
                                         'Latitude':'latitude',
                                         'Longitude':'longitude',
                                         'On Street Name':'on_street_name',
                                         'Off Street Name':'off_street_name',
                                         'Cross Street Name':'cross_street_name'
                                         }
                              )
                      .sort_values('date')
                      ) 
vehicle_collisions.head()


,date,time,collision_id,Town,latitude,longitude,zip_code,on_street_name,off_street_name,cross_street_name
37489,01/01/2022,1:30,4491344,NaN,40.819157,-73.96038,NaN,HENRY HUDSON PARKWAY,NaN,NaN
38026,01/01/2022,20:40,4491735,BRONX,40.824830,-73.85424,10473,NaN,2045 STORY AVENUE,NaN
38025,01/01/2022,20:35,4491457,MANHATTAN,40.741272,-73.97535,10016,EAST 30 STREET,NaN,1 AVENUE
38024,01/01/2022,19:50,4491408,NaN,40.759884,-73.93680,NaN,21 STREET,NaN,NaN
38022,01/01/2022,12:25,4491542,STATEN ISLAND,40.553314,-74.16340,10312,CORTELYOU AVENUE,NaN,GENESEE AVENUE


In [10]:
vehicle_collisions = (vehicle_collisions
                      .assign(date = lambda x: pd.to_datetime(x.date+' '+x.time))
                      )
vehicle_collisions.head()


,date,time,collision_id,Town,latitude,longitude,zip_code,on_street_name,off_street_name,cross_street_name
37489,2022-01-01 01:30:00,1:30,4491344,NaN,40.819157,-73.96038,NaN,HENRY HUDSON PARKWAY,NaN,NaN
38026,2022-01-01 20:40:00,20:40,4491735,BRONX,40.824830,-73.85424,10473,NaN,2045 STORY AVENUE,NaN
38025,2022-01-01 20:35:00,20:35,4491457,MANHATTAN,40.741272,-73.97535,10016,EAST 30 STREET,NaN,1 AVENUE
38024,2022-01-01 19:50:00,19:50,4491408,NaN,40.759884,-73.93680,NaN,21 STREET,NaN,NaN
38022,2022-01-01 12:25:00,12:25,4491542,STATEN ISLAND,40.553314,-74.16340,10312,CORTELYOU AVENUE,NaN,GENESEE AVENUE


In [11]:
## Deaths entries
deaths = (mvc_df[['Collision Id',
                  'Number Of Persons Killed',
                  'Number Of Pedestrians Killed',
                  'Number Of Cyclist Killed',
                  'Number Of Motorist Killed'
                  ]]
          .rename(columns={'Collision Id':'collision_id',
                           'Number Of Persons Killed': 'total',
                           'Number Of Pedestrians Killed': 'pedestrians',
                           'Number Of Cyclist Killed': 'cyclists',
                           'Number Of Motorist Killed': 'motorists'
                           }
                  )
          .sort_values('collision_id')
          )
deaths.info()


<class 'pandas.core.frame.DataFrame'>
Index: 200467 entries, 37493 to 2085624
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   collision_id  200467 non-null  int64  
 1   total         200467 non-null  float64
 2   pedestrians   200467 non-null  int64  
 3   cyclists      200467 non-null  int64  
 4   motorists     200467 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 9.2 MB


In [12]:
## Injuries entries
injuries = (mvc_df[['Collision Id', 
                    'Number Of Persons Injured',
                    'Number Of Pedestrians Injured',
                    'Number Of Cyclist Injured',
                    'Number Of Motorist Injured'
                    ]]
            .rename(columns={'Collision Id':'collision_id',
                             'Number Of Persons Injured': 'total',
                             'Number Of Pedestrians Injured': 'pedestrians',
                             'Number Of Cyclist Injured': 'cyclists',
                             'Number Of Motorist Injured': 'motorists'}
                    )
            .sort_values('collision_id')
            )
injuries.info()


<class 'pandas.core.frame.DataFrame'>
Index: 200467 entries, 37493 to 2085624
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   collision_id  200467 non-null  int64  
 1   total         200467 non-null  float64
 2   pedestrians   200467 non-null  int64  
 3   cyclists      200467 non-null  int64  
 4   motorists     200467 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 9.2 MB


In [13]:
## Causes entries
cause = (mvc_df[['Collision Id',
                 'Contributing Factor Vehicle 1',
                 'Contributing Factor Vehicle 2',
                 'Contributing Factor Vehicle 3',
                 'Contributing Factor Vehicle 4',
                 'Contributing Factor Vehicle 5'
                 ]]
         .rename(columns = {'Collision Id':'collision_id',
                            'Contributing Factor Vehicle 1':'vehicle_1',
                            'Contributing Factor Vehicle 2':'vehicle_2',
                            'Contributing Factor Vehicle 3':'vehicle_3',
                            'Contributing Factor Vehicle 4':'vehicle_4',
                            'Contributing Factor Vehicle 5':'vehicle_5'})
         .sort_values('collision_id')
         )
cause.info()


<class 'pandas.core.frame.DataFrame'>
Index: 200467 entries, 37493 to 2085624
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   collision_id  200467 non-null  int64 
 1   vehicle_1     199213 non-null  object
 2   vehicle_2     153774 non-null  object
 3   vehicle_3     18894 non-null   object
 4   vehicle_4     5012 non-null    object
 5   vehicle_5     1481 non-null    object
dtypes: int64(1), object(5)
memory usage: 10.7+ MB


In [14]:
## Vehicle types entries 
vehicle_type = (mvc_df[['Collision Id',
                        'Vehicle Type Code 1',
                        'Vehicle Type Code 2',
                        'Vehicle Type Code 3',
                        'Vehicle Type Code 4',
                        'Vehicle Type Code 5'
                       ]]
                .rename(columns = {'Collision Id':'collision_id',
                                   'Vehicle Type Code 1':'vehicle_1',
                                   'Vehicle Type Code 2':'vehicle_2',
                                   'Vehicle Type Code 3':'vehicle_3',
                                   'Vehicle Type Code 4':'vehicle_4',
                                   'Vehicle Type Code 5':'vehicle_5'})
                .sort_values('collision_id')
                )
vehicle_type.info()


<class 'pandas.core.frame.DataFrame'>
Index: 200467 entries, 37493 to 2085624
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   collision_id  200467 non-null  int64 
 1   vehicle_1     197448 non-null  object
 2   vehicle_2     132873 non-null  object
 3   vehicle_3     17515 non-null   object
 4   vehicle_4     4729 non-null    object
 5   vehicle_5     1414 non-null    object
dtypes: int64(1), object(5)
memory usage: 10.7+ MB


### **_Export extracted tables into the database and to `.csv`_**

In [15]:
# save to database
vehicle_collisions.to_sql('collisions', conn, if_exists='replace', index=False)
deaths.to_sql('deaths', conn, if_exists='replace', index=False)
injuries.to_sql('injuries', conn, if_exists='replace', index=False)
cause.to_sql('causes', conn, if_exists='replace', index=False)
vehicle_type.to_sql('vehicle_type', conn, if_exists='replace', index=False)


200467

In [16]:
# save to csv files
vehicle_collisions.to_csv('vehicle_collisions.csv', index=False)
deaths.to_csv('deaths.csv', index=False)
injuries.to_csv('injuries.csv', index=False)
cause.to_csv('causes.csv', index=False)
vehicle_type.to_csv('vehicle_type.csv', index=False)
